## Getting an access token


In [ ]:
import requests
import os

# Configuration
CLIENT_ID = os.environ.get('CLIENT_ID') # 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxx'
CLIENT_SECRET = os.environ.get('CLIENT_SECRET') # 'secret'

# Cache to store and reuse bearer token
token_cache = {
    "access_token": None
}

def fetch_access_token():
    try:
        response = requests.post(
            'https://api.starlink.com/auth/connect/token',
            headers={'Content-type': 'application/x-www-form-urlencoded'},
            data={
                'client_id': CLIENT_ID,
                'client_secret': CLIENT_SECRET,
                'grant_type': 'client_credentials',
            }
        )
        # Raise an exception for bad responses
        response.raise_for_status()

        # Process the response if successful
        response_data = response.json()
        access_token = response_data.get('access_token')

        if not access_token:
            raise ValueError("Access token not received in the response.")

        # Update the token cache
        token_cache["access_token"] = access_token

    except requests.RequestException as e:
        # Log the error along with the response body
        error_message = f"Failed to fetch access token: {str(e)}, Response body: {response.text}"
        print(error_message)
        raise Exception(error_message) from e

    except ValueError as e:
        error_message = f"Error processing the response: {str(e)}, Response body: {response.text}"
        print(error_message)
        raise Exception(error_message) from e

def get_header():
    if not token_cache["access_token"]:
        fetch_access_token()

    return {
        "Accept": "application/json",
        "Authorization": f'Bearer {token_cache["access_token"]}'
    }

def make_authorized_request(url, method, **kwargs):
    kwargs['headers'] = get_header()
    response = requests.request(method, url, **kwargs)

    if response.status_code == 401:
        # Token expired, fetch a new one and retry the request
        fetch_access_token()
        kwargs['headers'] = get_header()
        response = requests.request(method, url, **kwargs)

    return response

if __name__ == '__main__':
    # Example GET Request to get accounts.
    accounts_response = make_authorized_request('https://web-api.starlink.com/enterprise/v1/accounts', 'GET')
    print(accounts_response.json())

    # Example POST Request to get data usage.
    json = {
        "serviceLinesFilter": ["SL-123", "SL-456"],
        "previousBillingCycles": 1,
        "pageIndex": 0,
        "pageLimit": 50,
    }
    response = make_authorized_request('https://web-api.starlink.com/enterprise/v1/accounts/ACC-xxxxxxxxx-xxxxxx-xxxxx/billing-cycles/query', 'POST', json=json)
    print(response.json())

## Activating a Starlink

To activate a Starlink on a service line via API, follow these steps:

1. **POST an address**  
   Endpoint: [POST /enterprise/v1/account/{accountNumber}/addresses](https://starlink.readme.io/reference/post_enterprise-v1-account-accountnumber-addresses)

2. **GET available products**  
   Endpoint: [GET /enterprise/v1/account/{accountNumber}/service-lines/available-products](https://starlink.readme.io/reference/get_enterprise-v1-account-accountnumber-service-lines-available-products)

3. **POST a service line**  
   Endpoint: [POST /enterprise/v1/account/{accountNumber}/service-lines](https://starlink.readme.io/reference/post_enterprise-v1-account-accountnumber-service-lines)

4. **GET a user terminal ID from the paginated list**  
   Ensure this is the exact terminal being used at the address indicated in step 1.  
   Endpoint: [GET /enterprise/v1/account/{accountNumber}/user-terminals](https://starlink.readme.io/reference/get_enterprise-v1-account-accountnumber-user-terminals)

5. **POST a user terminal ID to the service line**  
   Endpoint: [POST /enterprise/v1/account/{accountNumber}/user-terminals/{userTerminalId}/service-line/{serviceLineNumber}](https://starlink.readme.io/reference/post_enterprise-v1-account-accountnumber-user-terminals-userterminalid-servicelinenumber)


### POST an address

In [ ]:
import requests

url = "https://web-api.starlink.com/enterprise/v1/account/accountNumber/addresses"

headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(url, headers=headers)

print(response.text)